The goal is to get all the paths from a directory (and its subdirs) in dropbox, then propose for each a new location and a new filename.


In [1]:
import sys
import dropbox
from dropbox.files import WriteMode
from dropbox.exceptions import ApiError, AuthError

from tokens import ACCESS_TOKEN

# Add OAuth2 access token here.
# You can generate one for yourself in the App Console.
# See <https://blogs.dropbox.com/developers/2014/05/generate-an-access-token-for-your-own-account/>
TOKEN = ACCESS_TOKEN


In [9]:
if (len(TOKEN) == 0):
    sys.exit("ERROR: Looks like you didn't add your access token. "
        "Open up backup-and-restore-example.py in a text editor and "
        "paste in your token in line 14.")

    # Create an instance of a Dropbox class, which can make requests to the API.
print("Creating a Dropbox object...")
with dropbox.Dropbox(TOKEN) as dbx:

    # Check that the access token is valid
    try:
        dbx.users_get_current_account()
    except AuthError:
        sys.exit("ERROR: Invalid access token; try re-generating an "
            "access token from the app console on the web.")
    
    

Creating a Dropbox object...


In [17]:
for entry in dbx.files_list_folder('/REEVOID_001_CT_NIFTI').entries:
    print(type(entry)==dropbox.files.FolderMetadata)
    print(entry.path_display)

True
/REEVOID_001_CT_NIFTI/_jsons
True
/REEVOID_001_CT_NIFTI/_localizer_others
True
/REEVOID_001_CT_NIFTI/_MRI
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_16.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_11.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_18.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_14.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_5.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_2.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_12.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_4.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_10.nii.gz
False
/REEVOID_001_CT_NIFTI/REEVOID_001_CT_DICOM_11_NATIF_CURML_20240517144156_8.nii.gz
Fal

In [19]:
def get_all_paths(dir):
    all_paths = []
    for entry in dbx.files_list_folder(dir).entries:
        if type(entry) == dropbox.files.FolderMetadata:
            all_paths += get_all_paths(entry.path_display)
        else:
            all_paths.append(entry.path_display)
    return all_paths

l = get_all_paths('/REEVOID_001_CT_NIFTI')

In [21]:
for path in l:
    print(path)

/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_1.1_NATIF_CURML_20240517144156_6.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_SURVEY_20240517155040_101_i00001.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_1.1_NATIF_CURML_20240517144156_17.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_1.1_NATIF_CURML_20240517144156_10.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_1.1_NATIF_CURML_20240517144156_1.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_B_FFE_COR_180_slices_20240517155040_301.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_3D_T1_TFE_SAG_lombaire_20240517155040_201.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_1.1_NATIF_CURML_20240517144156_8.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_T2_3D_TRA_Vista_20240517155040_501.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_SURVEY_20240517155040_101_i00019.json
/REEVOID_001_CT_NIFTI/_jsons/REEVOID_001_CT_DICOM_1.1_NATIF_CURML_20240517144156_4.json
/REEVOID_001